# Lets setup our workspace

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt2
import matplotlib.cm as cm
%matplotlib inline
from sklearn import preprocessing
from subprocess import check_output

Add **Data**

In [3]:
data = pd.read_csv("https://raw.githubusercontent.com/umeshpin/pyws01/main/bcdata.csv")

In [4]:
data = data.drop('id',axis=1)
data = data.drop('Unnamed: 32',axis=1)

# Mapping Benign to 0 and Malignant to 1

In [5]:
data['diagnosis'] = data['diagnosis'].map({'M':1,'B':0})

# Scaling the dataset



In [6]:
datas = pd.DataFrame(preprocessing.scale(data.iloc[:,1:32]))
datas.columns = list(data.iloc[:,1:32].columns)
datas['diagnosis'] = data['diagnosis']

# Creating the high dimensional feature space X

In [7]:
data_drop = datas.drop('diagnosis',axis=1)

X = data_drop.values

# Create a feed forward neural network with 3 hidden layers

In [13]:
from keras.models import Sequential, Model

from keras.layers import Dense, Dropout, Input

from tensorflow.keras.optimizers import SGD

model = Sequential()

model.add(Dense(128,activation="relu",input_dim = np.shape(X)[1]))

model.add(Dropout(0.25))

model.add(Dense(32, activation='relu'))

model.add(Dropout(0.25))

model.add(Dense(32, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


**Fit and test the model by randomly splitting it
67% of the data for training and 33% of the data for validation**

In [14]:
probas_ =model.fit(X, datas['diagnosis'], batch_size=5, epochs=10,validation_split=0.33)

Epoch 1/10
77/77 [==============================] - 2s 7ms/step - loss: 0.2805 - accuracy: 0.9003 - val_loss: 0.0614 - val_accuracy: 0.9894
Epoch 2/10
77/77 [==============================] - 0s 3ms/step - loss: 0.0914 - accuracy: 0.9738 - val_loss: 0.1017 - val_accuracy: 0.9574
Epoch 3/10
77/77 [==============================] - 0s 3ms/step - loss: 0.0956 - accuracy: 0.9633 - val_loss: 0.0603 - val_accuracy: 0.9787
Epoch 4/10
77/77 [==============================] - 0s 3ms/step - loss: 0.0836 - accuracy: 0.9738 - val_loss: 0.0653 - val_accuracy: 0.9840
Epoch 5/10
77/77 [==============================] - 0s 2ms/step - loss: 0.0767 - accuracy: 0.9738 - val_loss: 0.0443 - val_accuracy: 0.9787
Epoch 6/10
77/77 [==============================] - 0s 3ms/step - loss: 0.0551 - accuracy: 0.9790 - val_loss: 0.0486 - val_accuracy: 0.9840
Epoch 7/10
77/77 [==============================] - 0s 2ms/step - loss: 0.0905 - accuracy: 0.9685 - val_loss: 0.0614 - val_accuracy: 0.9787
Epoch 8/10
77/77 [==

# Cross validation analysis

In [15]:
from sklearn.model_selection import StratifiedKFold
# fix random seed for reproducibility
seed = 3
np.random.seed(seed)
# K fold cross validation (k=2)
k = 2
kfold = StratifiedKFold(n_splits=2, shuffle=True, random_state=seed)
cvscores = []
Y = datas['diagnosis']
for train, test in kfold.split(X, Y):
    # Fit the model
    model.fit(X[train], Y[train], epochs=10, batch_size=10, verbose=0)
    # evaluate the model
    scores = model.evaluate(X[test], Y[test], verbose=0)
    # Print scores from each cross validation run
    # print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    print("%d-fold cross validation accuracy -  %.2f%% (+/- %.2f%%)" % (k,np.mean(cvscores), np.std(cvscores)))

2-fold cross validation accuracy -  97.54% (+/- 0.00%)
2-fold cross validation accuracy -  98.07% (+/- 0.52%)


# Plot training & validation accuracy values